## MXenes

In [ ]:
import os
from ase import Atom
from ase.io import read
from ase.spacegroup import get_spacegroup

def identify_system_symmetry(atoms):
    symmetry = get_spacegroup(atoms, symprec=1e-5)
    international_symbol = symmetry.symbol
    spacegroup_number = symmetry.no
    return international_symbol, spacegroup_number

directory_path = "../examples/MXenes/"
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path) and file_path.endswith(".vasp"):
        atoms = read(file_path)
        international_symbol, spacegroup_number = identify_system_symmetry(atoms)
        print(f"File: {filename}, Symmetry: {international_symbol}, Spacegroup Number: {spacegroup_number}")
        print(atoms.cell.cellpar())

1. 根据 max_type, m_element, a_element, x_element, lattice_a, lattice_c 生成 MAX 体相结构；或者直接使用 max_atoms 导入 MAX 体相结构；

2. 删除 MAX 体相中的 A 层原子，根据 supercell_matrix, vacuum, n_layers, layer_distance 生成单层或多层的 MXene 二维结构；或者直接使用 mxene_atoms 导入单层 MXene 二维结构；

3. 使用 if_absorbed 控制是否考虑表面基团，并根据 adsorption_element, adsorption_sites_type, surface_type, adsorption_distance, coverage 在 MXene 表面添加表面基团；

4. MXenes 的 M 位可以被一个或多个过渡金属原子占据，形成固溶体或有序结构。
    有序的双过渡金属 MXenes 分为：
    1. 面内有序结构（i-MXenes），如(Mo2/3Y1/3)2CTx；
    2. 面内空位结构（如 W2/3CTx）；
    3. 面外有序结构（o-MXenes），其中一层 M" 过渡金属夹在两层 M' 过渡金属之间（如 Mo2TiC2Tx），或者两层 M" 过渡金属夹在两层 M' 过渡金属之间（如 Mo2Ti2C3Tx）。

补充 MAX 相化学计量比为 514 和 615 的MXene

In [ ]:
from ase import Atom, Atoms
from ase.build import mx2, molecule
from ase.data import chemical_symbols
from ase.io import read, write
from ase.visualize import view
from itertools import combinations
import numpy as np
import random

class MXeneBuilder:
    """
    用于搭建高对称的单过渡金属 MXene 2D结构的 python 类。
    """
    def __init__(self, max_type="211", m_element="Ti", a_element="Al", x_element="C", lattice_a=3.057, lattice_c=13.686, 
                 supercell_matrix=(3, 3, 1), vacuum=15.0, n_layers=1, layer_distance=5.0, 
                 if_absorbed=False, adsorption_atoms=None, adsorption_element='F', adsorption_sites_type="m_top", surface_type='both', adsorption_distance=2.5, coverage=1.0, **kwargs):
        self.max_type = max_type
        self.m_element = m_element
        self.a_element = a_element
        self.x_element = x_element
        self.lattice_a = lattice_a
        self.lattice_c = lattice_c
        
        self.supercell_matrix = supercell_matrix
        self.vacuum = vacuum
        self.n_layers = n_layers
        self.layer_distance = layer_distance
        
        self.if_absorbed = if_absorbed
        
        self.adsorption_atoms = adsorption_atoms
        self.adsorption_element = adsorption_element
        self.adsorption_sites_type = adsorption_sites_type
        self.surface_type = surface_type
        self.adsorption_distance = adsorption_distance
        self.coverage = coverage
        
        if self.layer_distance < self.adsorption_distance:
            raise ValueError(f"layer_distance ({self.layer_distance}) cannot be less than adsorption_distance ({self.adsorption_distance}).")
    
    def get_cell(self, a, c):
        cell = [[a, 0, 0],
                [- a / 2, a * np.sqrt(3) / 2, 0],
                [0, 0, c]]
        return cell
        
    def predefined_systems(self):
        predefined_max_dict = {
            "211": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 2 + [self.x_element] * 2,
                "cell": self.get_cell(self.lattice_a, self.lattice_c),
                "scaled_positions": [
                    [0.6666666666666666, 0.3333333333333333, 0.5840131100000000], # M1
                    [0.3333333333333333, 0.6666666666666666, 0.0840131099999999],
                    [0.3333333333333334, 0.6666666666666667, 0.4159868900000000], # M2
                    [0.6666666666666667, 0.3333333333333334, 0.9159868900000000],
                    [0.6666666666666666, 0.3333333333333333, 0.2500000000000000],
                    [0.3333333333333334, 0.6666666666666667, 0.7500000000000000],
                    [0.0000000000000000, 0.0000000000000000, 0.5000000000000000], # X
                    [0.0000000000000000, 0.0000000000000000, 0.0000000000000000],
                ],
                "m_top": {
                    "top": [[0.6666666666666666, 0.3333333333333333, 0.5840131100000000]],
                    "bottom": [[0.3333333333333334, 0.6666666666666667, 0.4159868900000000]],
                },
                "x_top": {
                    "top": [[0.0000000000000000, 0.0000000000000000, 0.5840131100000000]],
                    "bottom": [[0.0000000000000000, 0.0000000000000000, 0.4159868900000000]],
                },
                "hollow": {
                    "top": [[0.3333333333333334, 0.6666666666666667, 0.5840131100000000]],
                    "bottom": [[0.6666666666666666, 0.3333333333333333, 0.4159868900000000]],
                },
            },
            "312": {
                "symbols": [self.m_element] * 6 + [self.a_element] * 2 + [self.x_element] * 4,
                "cell": self.get_cell(self.lattice_a, self.lattice_c),
                "scaled_positions": [
                    [0.3333333333333334, 0.6666666666666666, 0.8726033900000001],
                    [0.6666666666666666, 0.3333333333333333, 0.3726033900000001], # M3
                    [0.6666666666666666, 0.3333333333333334, 0.1273966099999999],
                    [0.3333333333333334, 0.6666666666666667, 0.6273966099999999], # M1
                    [0.0000000000000000, 0.0000000000000000, 0.0000000000000000],
                    [0.0000000000000000, 0.0000000000000000, 0.4999999999999999], # M2
                    [0.0000000000000000, 0.0000000000000000, 0.2500000000000001],
                    [0.0000000000000000, 0.0000000000000000, 0.7500000000000000],
                    [0.6666666666666665, 0.3333333333333333, 0.9305387800000002],
                    [0.3333333333333334, 0.6666666666666666, 0.4305387800000003], # X2
                    [0.6666666666666666, 0.3333333333333334, 0.5694612199999997], # X1
                    [0.3333333333333335, 0.6666666666666667, 0.0694612199999998],
                ],
                "m_top": {
                    "top": [[0.3333333333333334, 0.6666666666666667, 0.6273966099999999]],
                    "bottom": [[0.6666666666666666, 0.3333333333333333, 0.3726033900000001]],
                },
                "x_top": {
                    "top": [[0.6666666666666666, 0.3333333333333334, 0.6273966099999999]],
                    "bottom": [[0.3333333333333334, 0.6666666666666666, 0.3726033900000001]],
                },
                "hollow": {
                    "top": [[0.0000000000000000, 0.0000000000000000, 0.6273966099999999]],
                    "bottom": [[0.0000000000000000, 0.0000000000000000, 0.3726033900000001]],
                },
            },
            "413": {
                "symbols": [self.m_element] * 8 + [self.a_element] * 2 + [self.x_element] * 6,
                "cell": self.get_cell(self.lattice_a, self.lattice_c),
                "scaled_positions": [
                    [0.6666666666666666, 0.3333333333333333, 0.9460444500000000],
                    [0.3333333333333334, 0.6666666666666667, 0.0539555499999999],
                    [0.0000000000000000, 0.0000000000000000, 0.1547523800000001],
                    [0.0000000000000000, 0.0000000000000000, 0.6547523800000001], # M1
                    [0.0000000000000000, 0.0000000000000000, 0.3452476199999998], # M4
                    [0.3333333333333333, 0.6666666666666666, 0.4460444499999999], # M3
                    [0.0000000000000000, 0.0000000000000000, 0.8452476199999999],
                    [0.6666666666666667, 0.3333333333333334, 0.5539555500000001], # M2
                    [0.3333333333333333, 0.6666666666666666, 0.2499999999999999],
                    [0.6666666666666667, 0.3333333333333334, 0.7500000000000000],
                    [0.3333333333333333, 0.6666666666666666, 0.6054759100000001], # X1
                    [0.6666666666666667, 0.3333333333333333, 0.1054759100000000],
                    [0.0000000000000000, 0.0000000000000000, 0.0000000000000000],
                    [0.3333333333333333, 0.6666666666666667, 0.8945240900000000],
                    [0.6666666666666667, 0.3333333333333334, 0.3945240899999999], # X3
                    [0.0000000000000000, 0.0000000000000000, 0.5000000000000001], # X2
                ],
                "m_top": {
                    "top": [[0.0000000000000000, 0.0000000000000000, 0.6547523800000001]],
                    "bottom": [[0.0000000000000000, 0.0000000000000000, 0.3452476199999998]],
                },
                "x_top": {
                    "top": [[0.3333333333333333, 0.6666666666666666, 0.6547523800000001]],
                    "bottom": [[0.6666666666666667, 0.3333333333333334, 0.3452476199999998]],
                },
                "hollow": {
                    "top": [[0.6666666666666667, 0.3333333333333334, 0.6547523800000001]],
                    "bottom": [[0.3333333333333333, 0.6666666666666666, 0.3452476199999998]],
                },
            },
        }
        return predefined_max_dict
        
    def _generate_max_structure(self):
        max_dict = self.predefined_systems()[self.max_type]
        structure = Atoms(symbols=max_dict["symbols"], 
                          scaled_positions=max_dict["scaled_positions"], 
                          cell=max_dict["cell"], pbc=True)
        return structure
    
    def _generate_single_mxene_structure(self):
        structure = self._generate_max_structure().copy()
        
        a_element_z_coords = [atom.position[2] for atom in structure if atom.symbol == self.a_element]
        min_a_element_z = min(a_element_z_coords)
        max_a_element_z = max(a_element_z_coords)    
        indices_to_remove = [
            atom.index for atom in structure
            if atom.symbol == self.a_element or atom.position[2] < min_a_element_z or atom.position[2] > max_a_element_z
        ]
        structure = structure[[atom.index for atom in structure if atom.index not in indices_to_remove]]

        return structure
        
    def _get_adsorption_sites(self):
        structure = self._generate_single_mxene_structure().copy()
        adsorption_sites_dict = self.predefined_systems()[self.max_type][self.adsorption_sites_type]

        adsorption_sites_dict['top'][0][2] += self.adsorption_distance / self.lattice_c
        adsorption_sites_dict['bottom'][0][2] -= self.adsorption_distance / self.lattice_c
        
        return adsorption_sites_dict
   
    def _generate_adsorption_atoms(self):
        if self.adsorption_atoms is None:
            if self.adsorption_element in chemical_symbols:
                adsorption_atoms = Atoms(self.adsorption_element)
            else:
                raise ValueError(f"Unsupported adsorption element: {self.adsorption_element}")
        else:
            adsorption_atoms = self.adsorption_atoms
        
        element_symbols = adsorption_atoms.get_chemical_symbols()
        unique_elements = set(element_symbols)
        
        return adsorption_atoms, unique_elements

    def _generate_adsorbed_single_mxene_structure(self):
        adsorption_sites = self._get_adsorption_sites()
        top_sites = adsorption_sites['top']
        bottom_sites = adsorption_sites['bottom']

        structure = self._generate_single_mxene_structure().copy()
        cell_matrix = structure.cell
        
        adsorption_atoms = self._generate_adsorption_atoms()[0].copy()
        com = adsorption_atoms.get_center_of_mass()

        def add_adsorption_atoms(sites, is_bottom=False):
            for site in sites:
                absolute_site = np.dot(site, cell_matrix)
                translation_vector = absolute_site - com
                adsorbed_atoms = adsorption_atoms.copy()
                
                if is_bottom:
                    adsorbed_atoms.rotate(180, 'x', center=com)
                
                adsorbed_atoms.translate(translation_vector)
                structure.extend(adsorbed_atoms)
    
        if self.surface_type in ['top', 'both']:
            add_adsorption_atoms(top_sites)
        if self.surface_type in ['bottom', 'both']:
            add_adsorption_atoms(bottom_sites, is_bottom=True)
    
        return structure
    
    def _generate_mxene_structure(self):
        if self.supercell_matrix[2] != 1:
            print(f"Warning: The third component of supercell_matrix should be 1, but got {self.supercell_matrix[2]}. It will be set to 1.")
            self.supercell_matrix = (self.supercell_matrix[0], self.supercell_matrix[1], 1)
            
        if self.if_absorbed == True:
            structure = self._generate_adsorbed_single_mxene_structure().copy() * self.supercell_matrix
        else:
            structure = self._generate_single_mxene_structure().copy() * self.supercell_matrix

        if self.n_layers > 1:
            multi_layer_structure = structure.copy()
            single_layer_height = max([atom.position[2] for atom in structure]) - min([atom.position[2] for atom in structure])
            for layer in range(1, self.n_layers):
                new_layer = structure.copy()
                for atom in new_layer:
                    atom.position[2] += layer * (single_layer_height + self.layer_distance)
                multi_layer_structure += new_layer
            structure = multi_layer_structure
            
        return structure
    
    def _generate_absorbed_mxene_structure(self):
        structure = self._generate_mxene_structure()
        adsorption_atoms, unique_elements = self._generate_adsorption_atoms()
            
        if self.if_absorbed:
            adsorption_molecule_indices = []
            for atom in structure:
                if atom.symbol in unique_elements:
                    adsorption_molecule_indices.append(atom.index)
                    
            num_adsorption_molecules = int(len(adsorption_molecule_indices) / len(adsorption_atoms))
            num_adsorption_sites_to_keep = int(num_adsorption_molecules * self.coverage)
            
            indices_to_keep = random.sample(
                range(num_adsorption_molecules), 
                num_adsorption_sites_to_keep
            )
            
            indices_to_remove = []
            adsorption_atoms_per_molecule = len(adsorption_atoms)
            for molecule_index in range(num_adsorption_molecules):
                if molecule_index not in indices_to_keep:
                    indices_to_remove.extend(
                        adsorption_molecule_indices[
                            molecule_index * adsorption_atoms_per_molecule : 
                            (molecule_index + 1) * adsorption_atoms_per_molecule
                        ]
                    )
                    
            del structure[indices_to_remove]
        
        min_z = min([atom.position[2] for atom in structure])
        max_z = max([atom.position[2] for atom in structure])
    
        z_length_with_vacuum = (max_z - min_z) + self.vacuum
        structure.set_cell([structure.cell[0], structure.cell[1], [0, 0, z_length_with_vacuum]])
        structure.center()
        
        return structure

In [ ]:
system = MXeneBuilder(max_type="211", m_element="W", a_element="Al", x_element="C", lattice_a=5, lattice_c=18, 
                      supercell_matrix=(3, 3, 1), vacuum=15.0, n_layers=1, layer_distance=5.0, 
                      if_absorbed=True, adsorption_atoms=read("../examples/MXenes/OH.vasp"), 
                      adsorption_element='F', adsorption_sites_type="hollow", surface_type='both', adsorption_distance=2.5, coverage=1.0)
atoms = system._generate_absorbed_mxene_structure()
# write("mxene.vasp", atoms, sort=True, vasp5=True)
view(atoms)

### Q-Api 接口

In [ ]:
def mxene_builder(**kwargs):
    kwargs['supercell_matrix'] = (kwargs['n_a'], kwargs['n_b'], 1)
    system = MXeneBuilder(**kwargs)
    new_structure = system._generate_absorbed_mxene_structure()
    cif_output = io.BytesIO()
    write(cif_output, new_structure, format='cif')
    cif_string = cif_output.getvalue().decode('utf-8')
    return cif_string

In [ ]:
@guide_register_func(ModelTag('构建高对称 MXene 材料结构', ['催化','电池'],['C'],nano_type='2d').identifier)
def MXene(data):
    mode = data.get('mode')
    if mode == 'init':
        title = Description(name='title', note='构建高对称 MXene 材料结构，需要指定 MAX 相基体元素及晶格参数等信息')
        max_type = SingleFromList(name='max_type', note='MAX 相类型', id='max_type', default_value='211', list_value=[{'label': str(i), 'value': str(i)} for i in [211, 312, 413]], is_required=1)
        m_element = SingleFromList(name='m_element', 
                                   note='MAX 相基体中的 M 元素', 
                                   id='m_element', 
                                   list_value=[{'label': '钛', 'value': 'Ti'},
                                               {'label': '钒', 'value': 'V' },
                                               {'label': '铬', 'value': 'Cr'},
                                               {'label': '锆', 'value': 'Zr'},
                                               {'label': '铌', 'value': 'Nb'},
                                               {'label': '钼', 'value': 'Mo'},
                                               {'label': '铪', 'value': 'Hf'},
                                               {'label': '钽', 'value': 'Ta'},
                                               {'label': '钨', 'value': 'W' }],
                                   is_required=1)
        a_element = SingleFromList(name='a_element', 
                                   note='MAX 相基体中的 A 元素', 
                                   id='a_element', 
                                   list_value=[{'label': '铝', 'value': 'Al'},
                                               {'label': '硅', 'value': 'Si'},
                                               {'label': '镓', 'value': 'Ga'},
                                               {'label': '锗', 'value': 'Ge'},
                                               {'label': '锡', 'value': 'Sn'}],
                                   is_required=1)
        x_element = SingleFromList(name='x_element', 
                                   note='MAX 相基体中的 X 元素', 
                                   id='x_element', 
                                   list_value=[{'label': '碳', 'value': 'C'},
                                               {'label': '氮', 'value': 'N' }],
                                   is_required=1)
        lattice_a = SingleInput(name='lattice_a', note='ab 方向晶格常数 (Å)', id='lattice_a', input_type='float', default_value=3.057, min=1.0, max=100.0, is_required=1)
        lattice_c = SingleInput(name='lattice_c', note='c 方向晶格常数 (Å)', id='lattice_c', input_type='float', default_value=13.686, min=1.0, max=100.0, is_required=1)
        n_a = SingleInput(name='n_a', note='a 轴的单胞重复单元', id='n_a', input_type='int', default_value=3, min=1, max=10, is_required=0)
        n_b = SingleInput(name='n_b', note='b 轴的单胞重复单元', id='n_b', input_type='int', default_value=3, min=1, max=10, is_required=0)
        vacuum = SingleInput(name='vacuum', note='真空层厚度 (Å)', id='vacuum', input_type='float', default_value=15.0, min=10.0, max=100.0, is_required=0)
        n_layers = SingleInput(name='n_layers', note='MXene 重复单元数', id='n_layers', input_type='int', default_value=1, min=1, max=10, is_required=0)
        layer_distance = SingleInput(name='layer_distance', note='MXene 层间距 (Å)', id='layer_distance', input_type='float', default_value=5.0, min=3.0, max=100.0, is_required=0)
        if_absorbed = SingleFromSwitch(name='if_absorbed', note='是否在 MXene 表面考虑吸附', id='if_absorbed', default_value=0)
        adsorption_atoms = StructureFromList(name='adsorption_atoms', note='选择 MXene 表面吸附的分子结构', id='adsorption_atoms', structure_type='crystal', output_format='cif')
        adsorption_element = SingleFromList(name='adsorption_element', 
                                            note='MXene 表面吸附元素', 
                                            id='adsorption_element', 
                                            list_value=[{'label': '氟', 'value': 'F'},
                                                        {'label': '氧', 'value': 'O'},
                                                        {'label': '氯', 'value': 'Cl'}],
                                            is_required=0)
        adsorption_sites_type = SingleFromList(name='adsorption_sites_type', 
                                               note='MXene 表面吸附位点类型', 
                                               id='adsorption_sites_type', 
                                               list_value=[{'label': 'M顶位', 'value': 'm_top'},
                                                           {'label': 'X顶位', 'value': 'x_top'},
                                                           {'label': 'M-X桥位', 'value': 'mx_bridge'}],
                                               is_required=0)
        surface_type = SingleFromList(name='surface_type', 
                                      note='MXene 吸附表面选择', 
                                      id='surface_type', 
                                      list_value=[{'label': '上表面', 'value': 'top'},
                                                  {'label': '下表面', 'value': 'bottom'},
                                                  {'label': '上下表面', 'value': 'both'}],
                                      is_required=0)
        adsorption_distance = SingleInput(name='adsorption_distance', note='吸附原子与 MXene 表面间距 (Å)', id='adsorption_distance', input_type='float', default_value=2.5, min=1.0, max=5.0, is_required=0)
        coverage = SingleInput(name='coverage', note='吸附原子表面覆盖率', id='coverage', input_type='float', default_value=1.0, min=0.0, max=1.0, is_required=0)
        return [title(), max_type(), m_element(), a_element(), x_element(), lattice_a(), lattice_c(), 
                n_a(), n_b(), vacuum(), n_layers(), layer_distance(), if_absorbed(), adsorption_atoms(), 
                adsorption_element(), adsorption_sites_type(), surface_type(), adsorption_distance(), coverage()]
    
    elif mode == 'generate':
        value = data.get('value')
        cif_string = mxene_builder(**value)
        return {'file_content': cif_string, 'file_format': 'cif'}
